#### Strategia d'investimento sui componenti dell'indice S&P 100 con back test - Short term reversal
https://www.youtube.com/watch?v=xpP048vfNrg

Strategia "long only" - Compriamo i titoli andati peggio nel mese precedente e li teniamo per 1 mese. I titoli "perdenti" ("loser stocks") sono identificati come quelli il cui rendimento si colloca nel decile peggiore.

Il backtest consiste nel confrontare la performance di questa strategia negli ultimi 12 anni con quella dell'indice S&P 100

In [41]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import datetime as dt
from pandas.tseries.offsets import MonthEnd
import pandas_datareader.data as web
import matplotlib.pyplot as plt

Ci procuriamo i ticker dei 100 titoli del S&P100 su Wikipedia

https://en.wikipedia.org/wiki/S%26P_100

In [42]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]

In [43]:
tickers.head()

,Symbol,Name,Sector
0,AAPL,Apple,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe,Information Technology


In [44]:
tickers = tickers.Symbol

Scarichiamo i prezzi

In [45]:
tickers

0      AAPL
1      ABBV
2       ABT
3       ACN
4      ADBE
       ... 
96       VZ
97      WBA
98      WFC
99      WMT
100     XOM
Name: Symbol, Length: 101, dtype: object

In [46]:
tickers.replace('BRK.B', 'BRK-B', inplace=True)

In [47]:
tickers.head(30)

0      AAPL
1      ABBV
2       ABT
3       ACN
4      ADBE
5       AIG
6       AMD
7      AMGN
8       AMT
9      AMZN
10     AVGO
11      AXP
12       BA
13      BAC
14       BK
15     BKNG
16      BLK
17      BMY
18    BRK-B
19        C
20      CAT
21     CHTR
22       CL
23    CMCSA
24      COF
25      COP
26     COST
27      CRM
28     CSCO
29      CVS
Name: Symbol, dtype: object

In [48]:
start_date = '2008-12-31'
end_date = datetime.today()

In [49]:
df = web.get_data_yahoo(tickers, start_date, end_date) 

In [50]:
df

Attributes   Adj Close                                                  \
Symbols           AAPL        ABBV         ABT         ACN        ADBE   
Date                                                                     
2008-12-31    2.598352         NaN   18.505180   25.209108   21.290001   
2009-01-02    2.762747         NaN   18.571064   25.885653   23.020000   
2009-01-05    2.879345         NaN   18.252071   26.000975   23.129999   
2009-01-06    2.831853         NaN   17.655691   25.678076   24.219999   
2009-01-07    2.770662         NaN   17.558601   25.970222   24.230000   
...                ...         ...         ...         ...         ...   
2022-11-23  151.070007  159.389999  106.019997  294.529999  335.779999   
2022-11-25  148.110001  159.619995  106.959999  296.399994  334.299988   
2022-11-28  144.220001  158.429993  105.389999  290.600006  328.970001   
2022-11-29  141.169998  158.199997  105.000000  287.980011  326.779999   
2022-11-30  141.390106  157.289993  104.029999  288.690002  331.279999   

Attributes                                                           ...  \
Symbols           AIG        AMD        AMGN         AMT       AMZN  ...   
Date                                                                 ...   
2008-12-31  21.207159   2.160000   43.289658   23.899532   2.564000  ...   
2009-01-02  22.828091   2.380000   44.219181   24.608690   2.718000  ...   
2009-01-05  22.422859   2.670000   44.713921   24.510876   2.703000  ...   
2009-01-06  23.503477   2.780000   43.731945   24.641293   2.868000  ...   
2009-01-07  22.152704   2.660000   43.626991   23.687593   2.810000  ...   
...               ...        ...         ...         ...        ...  ...   
2022-11-23  60.959999  76.400002  283.399994  218.250000  94.129997  ...   
2022-11-25  61.080002  75.139999  283.739990  220.080002  93.410004  ...   
2022-11-28  60.740002  73.190002  282.619995  213.899994  93.949997  ...   
2022-11-29  61.439999  73.389999  281.989990  217.220001  92.419998  ...   
2022-11-30  61.500000  73.455002  280.390015  215.880005  92.069702  ...   

Attributes     Volume                                                 \
Symbols           UNH         UNP        UPS         USB           V   
Date                                                                   
2008-12-31  5584600.0   6556200.0  3977400.0  10742900.0  16371600.0   
2009-01-02  4885900.0   8053600.0  4233200.0  10883600.0  13199200.0   
2009-01-05  8518900.0  12520800.0  3593500.0  14067200.0  16600000.0   
2009-01-06  7423000.0  13970400.0  4578400.0  16001700.0  32506400.0   
2009-01-07  8228400.0  15205600.0  5222100.0  16651800.0  28441600.0   
...               ...         ...        ...         ...         ...   
2022-11-23  2238300.0   2405000.0  1583500.0   6225700.0   3807300.0   
2022-11-25  1419200.0   1101900.0   912400.0   2564600.0   1993500.0   
2022-11-28  2699900.0   2755400.0  2043000.0   6790600.0  10229400.0   
2022-11-29  3751000.0   2280600.0  3670700.0   7854200.0   8986500.0   
2022-11-30  1247786.0   1061786.0  1115719.0   2629398.0   1666532.0   

Attributes                                                              
Symbols             VZ         WBA         WFC         WMT         XOM  
Date                                                                    
2008-12-31  17429248.0   6572600.0  45109700.0  13881900.0  30026400.0  
2009-01-02  14848985.0   5266700.0  36522300.0  16054800.0  35803700.0  
2009-01-05  36096596.0  11938400.0  43614200.0  16021300.0  43340100.0  
2009-01-06  28001608.0   7691300.0  54222900.0  19146000.0  41906100.0  
2009-01-07  22727743.0   8245300.0  52631000.0  16782000.0  35268800.0  
...                ...         ...         ...         ...         ...  
2022-11-23  14170000.0   3889500.0  10878500.0   5640200.0  11546700.0  
2022-11-25   7725700.0   2225500.0   4913300.0   2925800.0   7136000.0  
2022-11-28  21560900.0   4687900.0  12492600.0   5891600.0  23015500.0  
2022-11-29  1

In [51]:
prices = df['Adj Close']

In [52]:
prices

Symbols,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-31,2.598352,NaN,18.505180,25.209108,21.290001,21.207159,2.160000,43.289658,23.899532,2.564000,...,22.071682,18.112682,37.139252,18.330513,11.924665,16.824930,18.195292,20.919508,41.229034,48.728283
2009-01-02,2.762747,NaN,18.571064,25.885653,23.020000,22.828091,2.380000,44.219181,24.608690,2.718000,...,22.893147,18.995575,37.778881,18.506416,12.149747,17.192202,18.844339,21.288507,42.052746,49.833134
2009-01-05,2.879345,NaN,18.252071,26.000975,23.129999,22.422859,2.670000,44.713921,24.510876,2.703000,...,22.519754,19.723120,37.152725,17.597584,12.236136,16.120173,19.795774,19.911850,41.567348,49.827015
2009-01-06,2.831853,NaN,17.655691,25.678076,24.219999,23.503477,2.780000,43.731945,24.641293,2.868000,...,21.988703,20.465815,37.314304,17.407026,13.097808,15.862090,19.699890,19.542852,41.199612,49.015182
2009-01-07,2.770662,NaN,17.558601,25.970222,24.230000,22.152704,2.660000,43.626991,23.687593,2.810000,...,21.822752,18.855370,35.960968,17.128511,12.813615,16.063519,19.935907,18.357782,40.846611,47.763882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23,151.070007,159.389999,106.019997,294.529999,335.779999,60.959999,76.400002,283.399994,218.250000,94.129997,...,529.710022,212.520004,182.839996,44.110001,211.729996,38.919998,41.759998,47.330002,152.419998,113.610001
2022-11-25,148.110001,159.619995,106.959999,296.399994,334.299988,61.080002,75.139999,283.739990,220.080002,93.410004,...,537.619995,213.330002,182.520004,44.259998,213.789993,39.020000,41.660000,47.439999,153.070007,113.209999
2022-11-28,144.220001,158.429993,105.389999,290.600006,328.970001,60.740002,73.190002,282.619995,213.899994,93.949997,...,532.270020,208.520004,179.910004,43.740002,211.259995,38.240002,40.810001,46.980000,153.509995,109.809998


In [53]:
mtl_ret = prices.pct_change().resample('M').agg(lambda x: (x+1).prod() -1)

In [54]:
mtl_ret

Symbols,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2009-01-31,0.056005,0.000000,0.046311,-0.037511,-0.093001,-0.184713,0.013889,-0.050216,0.034788,0.147036,...,0.065038,-0.083891,-0.229696,-0.406637,-0.059104,-0.106012,0.111067,-0.358888,-0.159472,-0.041964
2009-02-28,-0.009098,0.000000,-0.146104,-0.075095,-0.135163,-0.671875,-0.004566,-0.107931,-0.040211,0.101496,...,-0.306389,-0.137344,-0.020515,-0.035714,0.151384,-0.044861,-0.125855,-0.347822,0.044991,-0.107712
2009-03-31,0.177024,0.000000,0.007604,-0.058239,0.280838,1.380952,0.399083,0.012059,0.044987,0.133508,...,0.066698,0.095682,0.195240,0.024038,-0.019573,0.058535,0.088014,0.176860,0.064051,0.002946
2009-04-30,0.197013,0.000000,-0.114598,0.070571,0.278635,0.379999,0.183607,-0.021204,0.043707,0.096405,...,0.123746,0.195329,0.063389,0.247091,0.168346,0.018825,0.210709,0.405197,-0.032629,-0.020999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-31,0.188634,-0.054242,0.006201,0.106967,0.120363,0.012517,0.235386,0.017139,0.059627,0.270596,...,0.055896,0.065735,0.067656,0.025641,0.077302,-0.089852,0.045383,0.119990,0.086116,0.131831
2022-08-31,-0.031208,-0.063062,-0.056872,-0.058121,-0.089437,-0.000386,-0.101620,-0.021474,-0.061958,-0.060615,...,-0.042427,-0.012275,-0.001950,-0.033686,-0.063175,-0.094826,-0.115093,-0.003647,0.003786,-0.013824
2022-09-30,-0.120977,-0.001859,-0.057379,-0.108022,-0.263068,-0.077063,-0.253446,-0.062006,-0.154891,-0.108622,...,-0.027516,-0.132243,-0.169503,-0.115983,-0.105984,-0.091844,-0.104393,-0.079844,-0.021501,-0.086620


In [55]:
mtl_ret.describe()

Symbols,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,...,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,0.027181,0.013381,0.011817,0.016639,0.019676,0.025584,0.035026,0.013228,0.014594,0.025343,...,0.021133,0.016893,0.012029,0.007995,0.019020,0.006144,0.008028,0.008983,0.009111,0.007274
std,0.079746,0.063321,0.054718,0.063861,0.080626,0.248114,0.169318,0.064696,0.053398,0.088826,...,0.063326,0.066338,0.069972,0.070819,0.060256,0.049489,0.081346,0.090081,0.051511,0.069953
min,-0.181198,-0.182854,-0.151885,-0.144864,-0.263068,-0.671875,-0.410489,-0.153768,-0.154891,-0.237525,...,-0.306389,-0.142455,-0.229696,-0.406637,-0.195747,-0.117070,-0.192985,-0.358888,-0.159472,-0.261858
25%,-0.021655,-0.001936,-0.019613,-0.024958,-0.037170,-0.048026,-0.070750,-0.031284,-0.019971,-0.037988,...,-0.019481,-0.018832,-0.020716,-0.025698,-0.022458,-0.021979,-0.048377,-0.031498,-0.023803,-0.025575
50%,0.029116,0.000000,0.009404,0.020486,0.028207,0.012258,0.016919,0.013372,0.010590,0.024735,...,0.021759,0.020248,0.009820,0.017486,0.023544,0.003281,-0.000794,0.013937,0.008942,0.003574
75%,0.078860,0.042927,0.045592,0.053687,0.063180,0.062173,0.152413,0.044760,0.048057,0.081428,...,0.062906,0.056726,0.048575,0.044882,0.054987,0.039141,0.057874,0.046331,0.042038,0.044023
max,0.216569,0.228907,0.171940,0.160929,0.280838,2.449771,0.523913,0.199423,0.161969,0.272601,...,0.178456,0.231570,0.284044,0.247091,0.168346,0.125896,0.260030,0.405197,0.147764,0.269156


In [56]:
formation = dt.datetime(2009,3,31)

In [57]:
formation

datetime.datetime(2009, 3, 31, 0, 0)

In [58]:
ret_1 = mtl_ret.loc[formation - MonthEnd(1)]

In [59]:
ret_1

Symbols
AAPL   -0.009098
ABBV    0.000000
ABT    -0.146104
ACN    -0.075095
ADBE   -0.135163
          ...   
VZ     -0.044861
WBA    -0.125855
WFC    -0.347822
WMT     0.044991
XOM    -0.107712
Name: 2009-02-28 00:00:00, Length: 101, dtype: float64

In [60]:
ret_1.name = 'mtl_ret'

In [61]:
ret_1 = pd.DataFrame(ret_1)

In [62]:
ret_1

,mtl_ret
Symbols,
AAPL,-0.009098
ABBV,0.000000
ABT,-0.146104
ACN,-0.075095
ADBE,-0.135163
...,...
VZ,-0.044861
WBA,-0.125855
WFC,-0.347822


In [64]:
ret_1['decile'] = pd.qcut(ret_1.mtl_ret,10,labels=False)

In [65]:
ret_1

,mtl_ret,decile
Symbols,,
AAPL,-0.009098,7
ABBV,0.000000,7
ABT,-0.146104,2
ACN,-0.075095,5
ADBE,-0.135163,2
...,...,...
VZ,-0.044861,5
WBA,-0.125855,3
WFC,-0.347822,0


In [66]:
losers = ret_1[ret_1.decile == 0].index

In [67]:
losers

Index(['AIG', 'AXP', 'BA', 'BAC', 'C', 'GD', 'GE', 'LMT', 'MET', 'UNH', 'WFC'], dtype='object', name='Symbols')

In [68]:
losret = mtl_ret.loc[formation, mtl_ret.columns.isin(losers)].mean()

Questo rendimento è il peggiore di tutto il campione. Se andiamo un mese avanti le cose migliorano

In [69]:
losret


0.3426155658958679

Simuliamo la strategia in tutto il periodo

In [70]:
def reversal(formation):
    ret_1 = mtl_ret.loc[formation - MonthEnd(1)]
    ret_1.name = 'mtl_ret'
    ret_1 = pd.DataFrame(ret_1)
    ret_1['decile'] = pd.qcut(ret_1.mtl_ret, 10, labels=False,duplicates = 'drop')
    losers = ret_1[ret_1.decile == 0].index
    losret = mtl_ret.loc[formation, mtl_ret.columns.isin(losers)].mean()
    return losret
    
    

In [71]:
reversal(formation)

0.3426155658958679

In [72]:
returns = []
dates = []

for i in mtl_ret.index[2:]:
    returns.append(reversal(i))
    dates.append(i)

In [73]:
returns

[-0.23231804696078984,
 0.3426155658958679,
 0.0491138514325431,
 0.05360088785582005,
 0.03520299784233216,
 0.06334119636673786,
 0.2255255698834805,
 0.05284290651859644,
 -0.020801298490794807,
 0.09031903577813656,
 0.009936506866433555,
 0.02302673969781598,
 0.032383811046209424,
 0.08116016057590476,
 -0.027089411400547193,
 -0.07336844102467165,
 -0.07674332315305138,
 0.07941561339841627,
 0.003998095330956672,
 0.10684559557825707,
 0.015071084574380304,
 0.014911101014318523,
 0.05148740024729461,
 0.022745106645408696,
 0.017120962402837427,
 -0.02259540149488974,
 0.04486932907991328,
 -0.021203795879822938,
 -0.008728951924190896,
 -0.04954619994631124,
 -0.08472528979949784,
 -0.14289195062764556,
 0.1364467599799431,
 -0.01860167076853185,
 -0.01945462371560019,
 0.10901707663612252,
 0.0636002112111545,
 0.03846212825939717,
 -0.033446387056296316,
 -0.119488495267079,
 0.04659840464247991,
 0.03668345757930307,
 0.06130360945488986,
 0.026916528890886936,
 0.00565180

In [74]:
frame =  pd.DataFrame({'dates': dates, 'returns':returns})

In [75]:
frame

,dates,returns
0,2009-02-28,-0.232318
1,2009-03-31,0.342616
2,2009-04-30,0.049114
3,2009-05-31,0.053601
4,2009-06-30,0.035203
...,...,...
161,2022-07-31,0.147424
162,2022-08-31,-0.044497
163,2022-09-30,-0.153228
164,2022-10-31,0.133425


In [76]:
frame.returns.mean()

0.01737163120527588

Confrontiamo la strategia con il rendimento del benchmark

In [77]:
df2 = web.get_data_yahoo('^OEX', start_date, end_date)['Adj Close'] 

In [78]:
df2

Date
2008-12-31     431.540009
2009-01-02     444.519989
2009-01-05     440.829987
2009-01-06     442.720001
2009-01-07     429.839996
                 ...     
2022-11-23    1802.869995
2022-11-25    1800.180054
2022-11-28    1773.849976
2022-11-29    1767.329956
2022-11-30    1766.619995
Name: Adj Close, Length: 3504, dtype: float64

In [79]:
bench_ret = df2.pct_change().resample('M').agg(lambda x: (x+1).prod() -1)

In [80]:
bench_ret

Date
2008-12-31    0.000000
2009-01-31   -0.093294
2009-02-28   -0.110075
2009-03-31    0.083685
2009-04-30    0.075924
                ...   
2022-07-31    0.092709
2022-08-31   -0.046479
2022-09-30   -0.095770
2022-10-31    0.070582
2022-11-30    0.015001
Freq: M, Name: Adj Close, Length: 168, dtype: float64

In [81]:
frame['S&P100 ret'] = bench_ret[2:].values

In [82]:
frame

,dates,returns,S&P100 ret
0,2009-02-28,-0.232318,-0.110075
1,2009-03-31,0.342616,0.083685
2,2009-04-30,0.049114,0.075924
3,2009-05-31,0.053601,0.053103
4,2009-06-30,0.035203,0.004912
...,...,...,...
161,2022-07-31,0.147424,0.092709
162,2022-08-31,-0.044497,-0.046479
163,2022-09-30,-0.153228,-0.095770
164,2022-10-31,0.133425,0.070582


In [83]:
frame.describe()

,returns,S&P100 ret
count,166.000000,166.000000
mean,0.017372,0.010058
std,0.067468,0.043296
min,-0.232775,-0.110075
25%,-0.011244,-0.009889
50%,0.019368,0.015065
75%,0.049091,0.036053
max,0.342616,0.126773


Calcoliamo i rendimenti composti

In [84]:
stra_adj_ret = frame.returns.mean() - 0.5*(frame.returns.std())**2
stra_adj_ret

0.015095691741427471

In [87]:
sp100_adj_ret = frame['S&P100 ret'].mean() - 0.5*(frame['S&P100 ret'].std())**2
sp100_adj_ret

0.009120393688083426

Calcoliamo quante volte la strategia batte il benchmark

In [89]:
ok = frame[frame.returns > frame['S&P100 ret']].shape[0]
ok

105

In [90]:
tutti = frame.shape[0]
tutti

166

In [91]:
hit_ratio = ok/tutti
hit_ratio

0.6325301204819277